In [1]:
import torch
from torch.linalg import norm
import nibabel as nib
import logging
import torch.nn as nn
import numpy as np



a = 0.5
b = 1 - a
EGv = 0.3745672075
ErChuPai = 2 / 3.141592653589793  



def joint_loss(sources, mag_norm, reference, m):
    #m = number of time steps
    #reference = group map
    #y3 = output
    #c = magnitude normalization
    #Cosy1 = torch.cosh(sources)
    #logCosy1 = torch.log(Cosy1)
    #EGy1 = logCosy1.mean()
    #Negama = EGy1 - EGv
    #Jy1 = (EGy1 - EGv)**2
    #KwDaoshu = ErChuPai * mag_norm * (1 / (1 + (mag_norm * Jy1)**2))
    
    
    
    
    #normalize weights for testing, normalize sources: check diff of sources@ref.t
    loss = -(a * ErChuPai * torch.arctan(mag_norm * nege(sources)) + b * (1 / m) * sources.t() @ reference)
    #loss = (a * KwDaoshu * 2 * Negama * EYgy + b * Simgrad.squeeze())
    return loss


torch.set_default_tensor_type(torch.DoubleTensor)


class gigICA(torch.nn.Module):
    #init_sources = y1, init_weights = 2c, mag_norm = c, m = m
    def __init__(self, init_sources, init_weights, dim, mag_norm, m):
        super().__init__()
        self.W = nn.Linear(init_weights.shape[0], 1)
        self.W.weight = nn.parameter.Parameter(init_weights.reshape([1,-1]))
        self.mag_norm = mag_norm 
        self.m = m

    def forward(self, X, reference, last_sources):
        sources = self.W(X)
        #Simgrad = (1 / self.m) * X @ reference.t()
        Eyr = 1/self.m * (sources.squeeze() @ reference.squeeze())
        
        
        Simgrad = (1 / self.m) * X.t() @ reference
        
        return sources
        
        
'''
iternum = 10
    
#wc = wc / norm(wc)
#y1 = wc.t() @ Y     
Cosy1 = torch.cosh(y1)
logCosy1 = torch.log(Cosy1)
EGy1 = logCosy1.mean()
Negama = EGy1 - EGv
#as input
#dim = y1.shape[0] if len(y1.shape) > 1 else y1.shape[0]
EYgy = (1 / m) * Y @ (torch.tanh(y1)).t()
Jy1 = (EGy1 - EGv)**2
KwDaoshu = ErChuPai * c * (1 / (1 + (c * Jy1)**2))
Simgrad = (1 / m) * Y @ reference.t()


Gradient calculation:

g = a * KwDaoshu * 2 * Negama * EYgy + b * Simgrad#.view(Simgrad.shape[0], 1)
g_norm = torch.sqrt(g.T@g)#torch.linalg.norm(g)
d = g / g_norm
wx = wc + Nemda * d #wc.view(wc.shape[0], 1)
wx = wx / norm(wx)
y3 = wx.t() @ Y


Loss computation:
#Loss term
PreObjValue = a * ErChuPai * torch.arctan(c * nege(y3)) + b * (1 / m) * y3 @ reference.t()
ObjValueChange = PreObjValue - IniObjValue
ftol = 0.02
dg = g.t() @ d
#In body of loop
ArmiCondiThr = Nemda * ftol * dg
if ObjValueChange < ArmiCondiThr:
    Nemda = Nemda / 2
    continue
if torch.allclose(wc.t() @ wx, torch.zeros(1), atol=1e-5):
    break
elif itertime == iternum:
    break
IniObjValue = PreObjValue
y1 = y3
wc = wx
itertime = itertime + 1
'''      
        
def gigicar(FmriMatr, ICRefMax):
    # Convert numpy arrays to PyTorch tensors
    FmriMatr = torch.tensor(FmriMatr, dtype=torch.float64)
    ICRefMax = torch.tensor(ICRefMax, dtype=torch.float64)

    # Extract dimensions
    n, m = FmriMatr.shape
    n2, m2 = ICRefMax.shape

    # Subtract mean from observed data
    #FmriMat = FmriMatr - FmriMatr.mean(dim=1, keepdim=True)
    FmriMat=FmriMatr - torch.tile(torch.mean(FmriMatr,1),(m,1)).T
    # Calculate covariance matrix
    CovFmri = (FmriMat @ FmriMat.t()) / m

    # Perform PCA reduction on signal
    D, E = torch.linalg.eig(CovFmri)

    #D = D[:, 0].real if len(D.shape) > 1 else D.real  # Extract real parts of eigenvalues
    EsICnum = ICRefMax.shape[0]
    D = D.real
    # Sort eigenvalues and eigenvectors
    index = D.argsort()
    eigenvalues = D[index]
    cols=E.shape[1]
    Esort=torch.zeros(E.shape)
    dsort=torch.zeros(eigenvalues.shape)
    for i in range(cols):
        Esort[:,i] = E[:,index[cols-i-1] ]
        dsort[i]   = D[index[cols-i-1] ]


    thr = 0  # Set your threshold value here
    numpc = (dsort > thr).sum()

    # Perform PCA for selected components
    Epart = Esort[:, :numpc]#.real
    dpart = dsort[:numpc]
    Lambda_part = torch.diag(dpart)#.real
    # Whitening source signal
    tmp = torch.sqrt(Lambda_part)
    Lambda_inv = torch.linalg.inv(torch.sqrt(Lambda_part)) 
    WhitenMatrix = Lambda_inv @ Epart.t()
    Y = WhitenMatrix @ FmriMat
    if thr<1e-10 and numpc<n:
        for i in range(Y.shape[0]):
            Y[i,:]=Y[i,:]/torch.std(Y[i,:])
    # Normalize source signal
    #Y = F.normalize(Y, dim=1)
    # Normalize reference signals
    ICRefMaxC=ICRefMax - torch.tile(torch.mean(ICRefMax,1), (m2, 1)).T
    ICRefMaxN=torch.zeros((EsICnum,m2))
    for i in range(EsICnum):
        ICRefMaxN[i,:]=ICRefMaxC[i,:]/torch.std(ICRefMaxC[i,:])
    #ICRefMaxN = (ICRefMax - ICRefMax.mean(dim=1, keepdim=True)) / ICRefMax.std(dim=1, keepdim=True)

    # Computing negentropy
    NegeEva = torch.zeros((EsICnum, 1))
    for i in range(EsICnum):
        NegeEva[i] = nege(ICRefMaxN[i, :])

    iternum = 10
    a = 0.5
    b = 1 - a
    EGv = 0.3745672075
    ErChuPai = 2 / 3.141592653589793

    ICOutMax = torch.zeros((EsICnum, m))
    logger.info("Starting with EGv=%f" % EGv)
    
    gradients = []
    for ICnum in range(2):
        logger.info('gigicar component: %d/%d' % (ICnum, EsICnum))
        reference = ICRefMaxN[ICnum, :]
        wc = (reference @ torch.linalg.pinv(Y)).t()
        wc = wc / norm(wc)
        print("wc")
        print(torch.mean(wc))
        last_sources = wc.t() @ Y
        EyrInitial = (1 / m) * (last_sources) @ reference.t()
        NegeInitial = nege(last_sources)
        mag_norm = (torch.tan((EyrInitial * 3.141592653589793) / 2)) / NegeInitial
        IniObjValue = a * ErChuPai * torch.arctan(mag_norm * NegeInitial) + b * EyrInitial

        itertime = 1
        Nemda = 1
        print("init sources")
        print(torch.mean(last_sources))
        GICA = gigICA(last_sources,wc,0,mag_norm,m)
        
        
        optimizer = torch.optim.SGD(GICA.parameters(), lr=.001)
        grrs = []
        for i in range(iternum):
            
            optimizer.zero_grad()
            sources = GICA(Y.t(),reference.reshape([-1,1]), last_sources.reshape([-1,1]))
            loss = joint_loss(sources, mag_norm, reference.reshape([-1,1]), m).sum()
            loss.backward()
            print("loss")
            print(loss)
            gtmp = GICA.W.weight.grad.cpu().numpy()[0]
            grrs.append(gtmp)
            optimizer.step()
            #Weight norm here...
            #GICA.W.weight = torch.nn.Parameter(GICA.W.weight/norm(GICA.W.weight))
            itertime = itertime + 1
        wx = GICA.W.state_dict()['weight']
        Source = wx @ Y
        gradients.append(np.array(grrs))
        ICOutMax[ICnum, :] = Source.squeeze()
    np.save(f'{output_dir}/{sub_id}_grads.npy',np.array(gradients))
    TCMax = (1 / m) * FmriMatr @ ICOutMax.t()
    return ICOutMax, TCMax


def nege(x):
    y = torch.log(torch.cosh(x))
    E1 = y.mean()
    E2 = 0.3745672075
    return (E1 - E2)**2


########################################################################
# SETUP LOGGER
########################################################################



DEFAULT_REFERENCE_FN = 'pooled_47.nii'
DEFAULT_EXAMPLE_FN = 'example.nii'
FORMAT = '%(asctime)-15s %(message)s'
logging.basicConfig(filename='pygigicar.log',level=logging.INFO)
logging.basicConfig(format=FORMAT)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)


logger = logging.getLogger('pygigicar')
logger.setLevel(logging.INFO)
# add ch to logger
logger.handlers = []
logger.addHandler(ch)


########################################################################
# CALL FUNCTIONS
########################################################################
'''
if len(sys.argv) != 6:
    print("Usage: python gigicar.py sub_id func_file out_dir mask_file template_file ")
    print(sys.argv)
    sys.exit()
'''
#sub_id = sys.argv[1]
sub_id = '000300655084'
sub_id = "001312269620"
#sub_id = 'test'
print(f"sub_id:{sub_id}")

#func_file = sys.argv[2]
func_file = '/data/qneuromark/Data/FBIRN/ZN_Neuromark/ZN_Prep_fMRI/'+sub_id+'/SM.nii'
print(f"func_file:{func_file}")

#output_dir = sys.argv[3]
output_dir = './out/'
print(f"output_dir:{output_dir}")

#mask_file = sys.argv[4]
mask_file = '/data/users2/jwardell1/nshor_docker/examples/fbirn-project/FBIRN/group_mean_masks/mask_resampled.nii'
print(f"mask_file:{mask_file}")

#template_file = sys.argv[5]
template_file = '/data/users2/jwardell1/ica-torch-gica/sa_script_work/gica/group_level_analysis/Neuromark_fMRI_1.0.nii'
print(f"template_file:{template_file}")

'''
if not os.path.isfile(func_file):
    print("Error: subject's preprocessed fMRI file not found.")
    sys.exit()


if not os.path.isdir(output_dir):
    print("Error: output dir not found.")
    sys.exit()


if not os.path.isfile(mask_file):
    print("Error: mask file not found.")
    sys.exit()


if not os.path.isfile(template_file):
    print("Error: template file not found.")
    sys.exit()

'''




# Load images
src_img = nib.load(func_file)
src_data = torch.tensor(src_img.get_fdata(), dtype=torch.float64)

ref_img = nib.load(template_file)
ref_data = torch.tensor(ref_img.get_fdata(), dtype=torch.float64)

mask_img = nib.load(mask_file)
mask_data = torch.tensor(mask_img.get_fdata(), dtype=torch.float64)

# Create idx tensor
idx = torch.nonzero(mask_data).t()

# Mask source and reference images
src_data = src_data[idx[0], idx[1], idx[2], :].t()
print(f'src_data.shape {src_data.shape}')

ref_data = ref_data[idx[0], idx[1], idx[2], :].t()
print(f'ref_data.shape {ref_data.shape}')

# Convert idx to numpy for compatibility with existing code
idx_np = idx.cpu().numpy()

# Continue with the rest of your PyTorch code...
ICOutMax, TCMax = gigicar(src_data, ref_data)


TCMax = TCMax.cpu().numpy()
# Save time courses file
tcfilename = f'{output_dir}/{sub_id}_TCMax_Torchified.npy'
np.save(tcfilename, TCMax)

# Reconstruct brain voxels
xdim, ydim, zdim = mask_data.shape
n_comp = ICOutMax.shape[0]
image_stack = torch.zeros((xdim, ydim, zdim, n_comp))

# Convert idx to numpy for compatibility with existing code
idx_np = idx.cpu().numpy()

image_stack[idx_np[0], idx_np[1], idx_np[2], :] = ICOutMax.t()

# Save as nifti
print("saving")
nifti_img = nib.Nifti1Image(image_stack.numpy(), affine=mask_img.get_qform())
nifti_img.header.set_sform(mask_img.header.get_sform(), code=mask_img.get_qform('code')[1])
nifti_img.header.set_qform(mask_img.header.get_qform(), code=mask_img.get_qform('code')[1])
nifti_file = f'{output_dir}/{sub_id}_ICOutMax_Torchified_SQ.nii.gz'
nib.save(nifti_img, nifti_file)

sub_id:001312269620
func_file:/data/qneuromark/Data/FBIRN/ZN_Neuromark/ZN_Prep_fMRI/001312269620/SM.nii
output_dir:./out/
mask_file:/data/users2/jwardell1/nshor_docker/examples/fbirn-project/FBIRN/group_mean_masks/mask_resampled.nii
template_file:/data/users2/jwardell1/ica-torch-gica/sa_script_work/gica/group_level_analysis/Neuromark_fMRI_1.0.nii
src_data.shape torch.Size([157, 58104])
ref_data.shape torch.Size([53, 58104])


/tmp/ipykernel_3944648/3472944891.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  FmriMatr = torch.tensor(FmriMatr, dtype=torch.float64)
/tmp/ipykernel_3944648/3472944891.py:111: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ICRefMax = torch.tensor(ICRefMax, dtype=torch.float64)
/tmp/ipykernel_3944648/3472944891.py:136: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  /pytorch/aten/src/ATen/native/Copy.cpp:240.)
  Esort[:,i] = E[:,index[cols-i-1] ]
2024-03-05 16:48:26,559 - pygigicar - INFO - Starting with EGv=0.374567
2024-03-05 16:48:26,567 - pygigicar - INFO - gigicar component: 0/53


wc
tensor(0.0029)
init sources
tensor(9.0831e-15)
loss
tensor(-0.7384, grad_fn=<SumBackward0>)
loss
tensor(-0.7386, grad_fn=<SumBackward0>)
loss
tensor(-0.7389, grad_fn=<SumBackward0>)
loss
tensor(-0.7391, grad_fn=<SumBackward0>)
loss
tensor(-0.7393, grad_fn=<SumBackward0>)
loss
tensor(-0.7395, grad_fn=<SumBackward0>)
loss
tensor(-0.7398, grad_fn=<SumBackward0>)
loss
tensor(-0.7400, grad_fn=<SumBackward0>)
loss
tensor(-0.7402, grad_fn=<SumBackward0>)
loss
tensor(-0.7404, grad_fn=<SumBackward0>)


NameError: name 'np' is not defined